In [1]:
#importações
import matplotlib.pyplot as plt
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from textblob import TextBlob

In [2]:
load_dotenv()

f'apiKey={os.getenv('NEWSAPIKEY')}')

SyntaxError: unmatched ')' (3412069080.py, line 3)

In [24]:
class AnaliseSentimento:
    def __init__(self, api_key, assunto, idioma):
        self.api_key = api_key
        self.assunto = assunto
        self.idioma = idioma
        self.url = None
        self.data = None
        self.news_df = None

    def faz_pesquisa(self):
        self.url = (
            'https://newsapi.org/v2/everything?'
            f'q={self.assunto}&'
            f'language={self.idioma}&'
            'sortBy=publishedAt&'
            f'apiKey={self.api_key}'
        )

    def get_response(self):
        if self.url is None:
            raise ValueError("A URL não foi definida. Execute 'faz_pesquisa()' antes.")
        response = requests.get(self.url)
        self.data = response.json()

    def extrai_title_description(self):
        if self.data is None:
            raise ValueError("Os dados ainda não foram baixados. Execute 'get_response()' antes.")
        articles = self.data.get('articles', [])
        self.news_df = pd.DataFrame([{
            'title': article.get('title'),
            'description': article.get('description')
        } for article in articles])

    def get_sentiment(self, text):
        if text:
            blob = TextBlob(text)
            return blob.sentiment.polarity
        return 0

    def calcula_sentimento(self):
        if self.news_df is None:
            raise ValueError("O DataFrame de notícias ainda não foi criado. Execute 'extrai_title_description()' antes.")
        self.news_df['sentiment'] = self.news_df['description'].apply(self.get_sentiment)

    def gera_grafico(self):
        if self.news_df is None or 'sentiment' not in self.news_df:
            raise ValueError("O sentimento ainda não foi calculado. Execute 'calcula_sentimento()' antes.")
        plt.figure(figsize=(8, 8))
        plt.hist(self.news_df['sentiment'], bins=20, color='skyblue')
        plt.title(f'Distribuição do Sentimento das Notícias sobre "{self.assunto}"')
        plt.xlabel('Polaridade')
        plt.ylabel('Número de Notícias')
        plt.grid(True)
        plt.show()